Transform number of trips made by duration in hours

In [ ]:
SELECT
    DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)/ 60 AS from_hour,
    DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)/ 60 + 1 AS to_hour,
    COUNT(1) AS number_of_trips
FROM
    OPENROWSET(
        BULK 'trip_data_green_parquet/**',
        FORMAT = 'PARQUET',
        DATA_SOURCE = 'nyc_taxi_data_raw'    

    ) AS [trip_data]
GROUP BY DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)/ 60 ,
          DATEDIFF(MINUTE, lpep_pickup_datetime, lpep_dropoff_datetime)/ 60 + 1
ORDER BY from_hour, to_hour;

Identify the percentage of cash and credit card trips by borough

In [8]:
WITH trip_data_cte AS(
SELECT
   *
FROM
    OPENROWSET(
                BULK 'trip_data_green_parquet/year=2020/month=01/',
                FORMAT = 'PARQUET',
                DATA_SOURCE = 'nyc_taxi_data_raw'    

                ) AS [trip_data]
),

taxi_zone_cte AS(
SELECT *
FROM
    OPENROWSET(
                BULK 'taxi_zone.csv',
                DATA_SOURCE = 'nyc_taxi_data_raw',
                FORMAT = 'CSV',
                PARSER_VERSION = '2.0',
                FIRSTROW = 2
            ) 
            WITH (
                location_id SMALLINT 1,
                borough VARCHAR(15) 2,
                zone VARCHAR(50) 3,
                service_zone  VARCHAR(15) 4 
            )AS [taxi_zone]
),

payment_type_cte AS (
SELECT * FROM    
    OPENROWSET(
        BULK 'payment_type.json',
        DATA_SOURCE = 'nyc_taxi_data_raw',
        FORMAT = 'CSV',
        FIELDTERMINATOR = '0x0b',
        FIELDQUOTE = '0x0b'
    ) WITH(
        jsonDoc NVARCHAR(MAX)
    )AS payment_type
    CROSS APPLY OPENJSON(jsonDoc) --INNER JOIN WITHOUT ANY JOIN CONDITION.
    WITH(
        payment_type SMALLINT,
        description VARCHAR(20) '$.payment_type_desc'
    )
)  
SELECT

taxi_zone_cte.borough,
COUNT(1) AS total_number_of_trips,
SUM(CASE WHEN payment_type_cte.description = 'Cash' THEN 1 ELSE 0 END ) AS cash_trips,
SUM(CASE WHEN payment_type_cte.description = 'Credit card' THEN 1 ELSE 0 END ) AS crds_trips,
CAST((SUM(CASE WHEN payment_type_cte.description = 'Cash' THEN 1 ELSE 0 END )/ CAST(COUNT(1) AS DECIMAL)) * 100 AS decimal(5,2)) AS cash_percentage,
CAST((SUM(CASE WHEN payment_type_cte.description = 'Credit card' THEN 1 ELSE 0 END )/ CAST(COUNT(1) AS DECIMAL)) * 100 AS decimal(5,2)) AS card_percentage

FROM trip_data_cte
LEFT JOIN taxi_zone_cte ON trip_data_cte.PULocationID = taxi_zone_cte.location_id
LEFT JOIN payment_type_cte ON trip_data_cte.payment_type =  payment_type_cte.payment_type

WHERE payment_type_cte.description IN ('Cash', 'Credit Card')
GROUP BY taxi_zone_cte.borough
ORDER BY taxi_zone_cte.borough;

Statement ID: {193CFCE1-35EB-4CBA-95D6-6471CACE8223} | Query hash: 0x484779F84D218DA9 | Distributed request ID: {CDA992C1-1458-4CD2-B48D-9FC9848747F0}. Total size of data scanned is 2 megabytes, total size of data moved is 1 megabytes, total size of data written is 0 megabytes.

(7 rows affected)

Total execution time: 00:00:01.569

borough,total_number_of_trips,cash_trips,crds_trips,cash_percentage,card_percentage
Bronx,12116,5330,6786,43.99,56.01
Brooklyn,79997,25821,54176,32.28,67.72
EWR,4,0,4,0.00,100.00
Manhattan,137808,60543,77265,43.93,56.07
Queens,98648,53081,45567,53.81,46.19
Staten Island,138,31,107,22.46,77.54
Unknown,720,221,499,30.69,69.31
